In [1]:
import pandas as pd
import numpy as np

In [2]:
product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']

In [3]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')
train = train.loc[train['fecha_dato']=='2016-05-28']
lastmonthlabels = train[product_cols]
lastmonthlabels['ncodpers'] = train.ncodpers #check if another last month
lastmonthlabels = lastmonthlabels.set_index('ncodpers')
stacked_lastmonthlabels = lastmonthlabels.stack()
filtered_lastmonthlabels = stacked_lastmonthlabels.reset_index()
filtered_lastmonthlabels.columns = ["ncodpers", "product", "is_owned"]
lastmonthlabels=[]

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split
reader = Reader(line_format='user item rating', rating_scale=(0,1))
data = Dataset.load_from_df(filtered_lastmonthlabels, reader=reader)
trainset = data.build_full_trainset()
algo = SVDpp(verbose=True)
algo.fit(trainset)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19


NameError: name 'testset' is not defined

In [5]:
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
users = test.ncodpers

In [7]:
uid = users[0]
iid = product_cols[2]

In [8]:
pred =algo.predict(uid,iid)

In [9]:
pred[3]

0.7171903153687742

In [10]:
preds =[]
user_preds = []
for user in users:
    for product in product_cols:
        user_preds.append(algo.predict(user,product)[3])
    preds.append(user_preds)
    user_preds = []

In [11]:
preds =np.array([np.array(predi) for predi in preds])

In [12]:
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)

In [13]:
preds[0]

array([ 2,  4, 20, 16, 15, 19, 12,  5,  6, 21,  3, 13, 22, 23,  0,  1, 18,
       14, 11,  9,  8,  7, 17, 10], dtype=int64)

In [14]:
final_month_training_cols = train
final_month_training_cols = final_month_training_cols.drop(product_cols, axis=1)

In [15]:
#Get the target columns
labels_final_month = train[product_cols]

#Add ncodpers to the dataframe
labels_final_month['ncodpers'] = train.ncodpers

labels_final_month = labels_final_month.set_index("ncodpers")

stacked_labels_final_month = labels_final_month.stack()

filtered_labels_final_month = stacked_labels_final_month.reset_index()

filtered_labels_final_month.columns = ["ncodpers", "product", "newly_added"]

#Only select the rows where there is a new product.
filtered_labels_final_month = filtered_labels_final_month[filtered_labels_final_month["newly_added"] == 1]

#Merge with the training features.
multiclass_final_month = filtered_labels_final_month.merge(final_month_training_cols, on="ncodpers", how="left")

final_month = multiclass_final_month.drop_duplicates(multiclass_final_month, keep='last')

labels_final_month_final = final_month['product']

final_month_ncodpers = final_month.ncodpers
#Remove the columns that are not needed to train the model.
final_month = final_month.drop(['ncodpers','newly_added','product'], axis = 1)

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [16]:
labels_final_month_final_cat = labels_final_month_final.astype('category').cat.codes

In [17]:
used_products = pd.DataFrame()
used_products['product'] = labels_final_month_final_cat
used_products['ncodpers'] = final_month_ncodpers
used_products = used_products.drop_duplicates(keep = 'last')

In [18]:
#create a dictionary to store each product a customer already has
used_recommendation_products = {}
product_cols = np.array(product_cols)
#iterate through used_products and add each one to used_recommendation_products
for idx,row_val in used_products.iterrows():
    used_recommendation_products.setdefault(row_val['ncodpers'],[]).append(product_cols[row_val['product']])
    if len(used_recommendation_products) % 100000 == 0:
        print(len(used_recommendation_products))

100000
200000
300000
300000
400000
500000
600000
600000
600000
600000
600000
600000


In [19]:
#test_ids are the customer codes for the testing data.
test_ids = np.array(pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv',usecols=['ncodpers'])['ncodpers'])
target_cols_all = np.array(product_cols)
final_preds = []
#iterate through our model's predictions (pred) and add the 7 most recommended products that the customer does not have.
for idx,predicted in enumerate(preds):
    ids = test_ids[idx]
    top_product = target_cols_all[predicted]
    used_products = used_recommendation_products.get(ids,[])
    new_top_product = []
    for product in top_product:
        if product not in used_products:
            new_top_product.append(product)
        if len(new_top_product) == 7:
            break
    final_preds.append(' '.join(new_top_product))
    if len(final_preds) % 100000 == 0:
        print(len(final_preds))

100000
200000
300000
400000
500000
600000
700000
800000
900000


In [20]:
submission = pd.DataFrame({'ncodpers':test_ids,'added_products':final_preds})
submission.to_csv('submission.csv',index=False)

In [21]:
submission.shape

(929615, 2)

In [22]:
submission.head()

,added_products,ncodpers
0,ind_cno_fin_ult1 ind_recibo_ult1 ind_nomina_ul...,15889
1,ind_recibo_ult1 ind_cno_fin_ult1 ind_nom_pens_...,1170544
2,ind_recibo_ult1 ind_cno_fin_ult1 ind_nom_pens_...,1170545
3,ind_recibo_ult1 ind_cno_fin_ult1 ind_nom_pens_...,1170547
4,ind_recibo_ult1 ind_cno_fin_ult1 ind_nom_pens_...,1170548
